#### USCRN Data 

In [154]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import yaml 

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file)

/tmp/ipykernel_41028/798438717.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sources = yaml.load(yaml_file)


In [145]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")

In [51]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [130]:
import re

links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [ ]:
for url in file_urls:
  

In [133]:
file_response = requests.get(file_urls[0])

In [170]:
soup = BeautifulSoup(file_response.content, 'html.parser')
rows = str(soup).split('\n')
rows = [re.split(' +', row) for row in rows]

In [173]:
df = pd.DataFrame(rows, columns=columns)
df

,WBANNO,UTC_DATE,UTC_TIME,LST_DATE,LST_TIME,CRX_VN,LONGITUDE,LATITUDE,T_CALC,T_HR_AVG,...,SOIL_MOISTURE_5,SOIL_MOISTURE_10,SOIL_MOISTURE_20,SOIL_MOISTURE_50,SOIL_MOISTURE_100,SOIL_TEMP_5,SOIL_TEMP_10,SOIL_TEMP_20,SOIL_TEMP_50,SOIL_TEMP_100
0,26494,20020809,2200,20020809,1300,1.001,-147.51,64.97,-9999.0,-9999.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,26494,20020809,2300,20020809,1400,1.001,-147.51,64.97,10.0,11.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,26494,20020810,0000,20020809,1500,1.001,-147.51,64.97,12.0,10.5,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,26494,20020810,0100,20020809,1600,1.001,-147.51,64.97,12.1,11.9,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,26494,20020810,0200,20020809,1700,1.001,-147.51,64.97,11.9,12.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,26494,20021231,2100,20021231,1200,1.001,-147.51,64.97,-26.1,-26.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3456,26494,20021231,2200,20021231,1300,1.001,-147.51,64.97,-25.7,-25.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3457,26494,20021231,2300,20021231,1400,1.001,-147.51,64.97,-26.2,-26.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3458,26494,20030101,0000,20021231,1500,1.001,-147.51,64.97,-26.3,-26.3,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [92]:
import re
x = year_soups[1].find_all('a', href=re.compile(r'.*NC.*\.txt'))
x

[<a href="CRNH0203-2001-NC_Asheville_8_SSW.txt">CRNH0203-2001-NC_Asheville_8_SSW.txt</a>,
 <a href="CRNH0203-2001-NC_Asheville_13_S.txt">CRNH0203-2001-NC_Asheville_13_S.txt</a>]

In [99]:
{a.strip("_") for a in re.findall(r'_[A-Z]{1}[a-z]+_',x[0].getText())}

{'Asheville'}